In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
os.chdir('/home/s2316002/capstone_project/cic/dataset/Original_dataset')

Matplotlib created a temporary cache directory at /tmp/matplotlib-l8b_vorb because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

In [2]:
df1=pd.read_csv("Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("Wednesday-workingHours.pcap_ISCX.csv")


df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

nRow, nCol = df.shape
print(f'The table has {nRow} rows and {nCol} columns')

The table has 2830743 rows and 79 columns


In [3]:
def categorize_port(port):
    if 0 <= port <= 1023:
        return 1
    elif 1024 <= port <= 49151:
        return 2
    elif 49152 <= port <= 65535:
        return 3
    else:
        return None  # or however you want to handle out-of-range values

In [4]:
col_list = []
for i, name in enumerate(df.columns):
    if name[0] == ' ':
        name = name[1:]
    if ' ' in name:
        name = name.replace(' ', '_')
    col_list.append(name)
df.columns = col_list

In [5]:
df['Destination_Port'] = df['Destination_Port'].apply(categorize_port)

In [6]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
cols_with_nan = df.columns[df.isna().any()].tolist()
print("Columns containing NaN values:", cols_with_nan)


Columns containing NaN values: []


In [7]:
list = df.columns.tolist()
list = [x.lower() for x in list]
df.columns = list
df = df.rename(columns = {'destination_port': 'destination_port_priority'})

In [8]:

# One-hot encode the 'destination_port_priority' column
temp = pd.get_dummies(df['destination_port_priority'], prefix='destination_port_priority', dtype=int)

# Drop the original 'destination_port_priority' column and append the dummies
df = df.drop('destination_port_priority', axis=1).join(temp)

In [9]:
df

,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,...,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,destination_port_priority_1,destination_port_priority_2,destination_port_priority_3
0,3,2,0,12,0,6,6,6.0,0.00000,0,...,0,0,0.0,0.0,0,0,BENIGN,0,0,1
0,3,2,0,12,0,6,6,6.0,0.00000,0,...,0,0,0.0,0.0,0,0,BENIGN,1,0,0
0,3,2,0,12,0,6,6,6.0,0.00000,0,...,0,0,0.0,0.0,0,0,BENIGN,0,1,0
0,3,2,0,12,0,6,6,6.0,0.00000,0,...,0,0,0.0,0.0,0,0,BENIGN,0,0,1
0,3,2,0,12,0,6,6,6.0,0.00000,0,...,0,0,0.0,0.0,0,0,BENIGN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,32215,4,2,112,152,28,28,28.0,0.00000,76,...,0,0,0.0,0.0,0,0,BENIGN,1,0,0
692699,324,2,2,84,362,42,42,42.0,0.00000,181,...,0,0,0.0,0.0,0,0,BENIGN,1,0,0
692700,82,2,1,31,6,31,0,15.5,21.92031,6,...,0,0,0.0,0.0,0,0,BENIGN,0,0,1
692701,1048635,6,2,192,256,32,32,32.0,0.00000,128,...,0,0,0.0,0.0,0,0,BENIGN,1,0,0


In [10]:
df.to_csv('CIC_IDS2017.csv', index=False)

In [ ]:
df.label